In [32]:
import numpy as np
import matplotlib.pyplot as plt

In [33]:
def GPR_forward(sources, porosity, boreholes_distance):
    # Compute distance between sources
    dh = sources[1] - sources[0]
    # Create receivers array
    receivers = sources.copy()
    # Transform porosity into slowness vector
    slowness = porosity_to_slowness(porosity)
    times = np.zeros((6, 6))
    for i in range(6):
        for j in range(6):
            if i == j:
                # Fill the diagonal of times matrix
                times[i, i] = slowness[i] * boreholes_distance
            else:
                # Fill no diagonal elements
                height = abs(sources[i] - receivers[j])
                factor = (
                    dh
                    * np.sqrt(1 + boreholes_distance ** 2 / height **2)
                )
                times[i, j] = 0.5 * slowness[i]
                times[i, j] += 0.5 * slowness[j]
                min_index, max_index = min(i, j), max(i, j)
                times[i, j] += sum(slowness[min_index + 1 : max_index])
                times[i, j] *= factor
    return times


def porosity_to_slowness(porosity, kappa_s=5, kappa_w=81):
    c = 0.3  # speed of light in vacumm in m/ns
    porosity = np.array(porosity)
    kappa_sqrt = (1 - porosity) * np.sqrt(kappa_s) + porosity * np.sqrt(kappa_w)
    return kappa_sqrt / c


def define_porosity_layers(porosity):
    # Make 6 layers out of 4 porosity parameters
    porosity_layers = porosity.copy()
    porosity_layers.insert(1, porosity[0])
    porosity_layers.insert(3, porosity[2])
    return porosity_layers

In [34]:
# Define depths to sources and receivers (must be the same)
sources = np.linspace(0.5, 5.5, 6)  # in meters
boreholes_distance = 4  # in meters

In [39]:
porosity = define_porosity_layers([0, 1, 0, 0])
np.array(GPR_forward(sources, porosity, boreholes_distance), dtype=int)

array([[ 29,  30,  58,  74,  74,  76],
       [ 30,  29,  77,  83,  74,  74],
       [ 58,  77, 120,  77,  58,  56],
       [ 74,  83,  77,  29,  30,  33],
       [ 74,  74,  58,  30,  29,  30],
       [ 76,  74,  56,  33,  30,  29]])